In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 47.6 MB/s eta 0:00:00


In [4]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

#
model = DiscreteBayesianNetwork(
    [('Lluvia', 'Trafico'),
     ('Trafico', 'Tardanza')]
    )

# Definir las Tablas de Probabilidad Condicional (CPD)
# CPD Lluvia: P(Lluvia)
cpd_lluvia = TabularCPD(variable='Lluvia', variable_card=2, values=[[0.8], [0.2]]) # [No, Si]

# CPD Trafico: P(Trafico | Lluvia)
# Si no llueve (0), hay tráfico (1) con prob 0.1. Si llueve (1), hay tráfico (1) con prob 0.6.
cpd_trafico = TabularCPD(variable='Trafico', variable_card=2,
                         values=[[0.9, 0.4],  # No hay tráfico
                                 [0.1, 0.6]], # Sí hay tráfico
                         evidence=['Lluvia'], evidence_card=[2])

# CPD Tardanza: P(Tardanza | Trafico)
cpd_tardanza = TabularCPD(variable='Tardanza', variable_card=2,
                          values=[[0.9, 0.2],  # No llega tarde
                                  [0.1, 0.8]], # Sí llega tarde
                          evidence=['Trafico'], evidence_card=[2])

# 3. Asociar las tablas al modelo
model.add_cpds(cpd_lluvia, cpd_trafico, cpd_tardanza)

# Validar si el modelo es consistente (suma de probabilidades = 1)
print(f"¿Modelo válido?: {model.check_model()}")

# ¿Cuál es la probabilidad de llegar tarde si sabemos que ESTÁ LLOVIENDO?
infer = VariableElimination(model)
query = infer.query(variables=['Tardanza'], evidence={'Lluvia': 1})
print("\nProbabilidad de Tardanza dado que Llueve:")
print(query)

¿Modelo válido?: True

Probabilidad de Tardanza dado que Llueve:
+-------------+-----------------+
| Tardanza    |   phi(Tardanza) |
+=============+=================+
| Tardanza(0) |          0.4800 |
+-------------+-----------------+
| Tardanza(1) |          0.5200 |
+-------------+-----------------+
